In [1]:
import selenium
import re 
import json
import grequests
import requests
from bs4 import BeautifulSoup
from datetime import datetime 
import os
import pandas as pd
import shutil
import yaml
import pickle
#from helium import *
import time
import traceback
from agenda_tools import get_schedule, download_images, get_category
from requests_html import HTMLSession
from IPython.display import Image
import urllib

session = HTMLSession()

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from agenda_tools.get_schedule import remove_blank_spaces
from selenium.webdriver.common.keys import Keys

In [44]:
from selenium import webdriver


options = webdriver.FirefoxOptions()
options.add_argument('--private')
options.add_argument('--no-sandbox')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 12.2; rv:97.0) Gecko/20100101 Firefox/97.0')
#options.add_argument("--headless")
driver = webdriver.Firefox(executable_path='/home/cesar/Documents/job/web_scraping/javier/agenda/driver/geckodriver', options=options)

In [202]:
#url = 'https://www.booking.com/index.es.html?aid=397594&label=gog235jc-1DCAEoggI46AdIClgDaPEBiAEBmAEKuAEZyAEP2AED6AEB-AECiAIBqAIDuAKzgaiRBsACAdICJGZkNDVjNTExLTZkM2EtNDhiNS1hY2Q3LTg1MTI5MGVmOTMzONgCBOACAQ&sid=d66a016dcaeb9f47c6acc13833910118&srpvid=b41084be967b0215&click_from_logo=1'
driver.get(url)

## Habitaciones

In [141]:
habitaciones = [i.text for i in driver.find_elements_by_xpath('//tbody/tr/td[@class="hprt-table-cell -first hprt-table-cell-roomtype droom_seperator"]//div[@class="hprt-roomtype-block hprt-roomtype-name hp_rt_room_name_icon__container"]')]
habitaciones

['Habitación Doble - 2 camas', 'Habitación Triple', 'Habitación cuádruple']

## Tipo de comas

In [140]:
camas_types = [i.text for i in driver.find_elements_by_xpath('//tbody/tr/td[@class="hprt-table-cell -first hprt-table-cell-roomtype droom_seperator"]//div[@class="hprt-roomtype-bed"]')]
camas_types

['2 camas individuales',
 '1 cama individual y 1 cama doble',
 '4 camas individuales']

In [86]:
len_rows = len(driver.find_elements_by_xpath(f'//tbody/tr'))

In [98]:
list_services = []
for i in range(1,len_rows):
     list_services.append([w.text for w in driver.find_elements_by_xpath(f'//tbody/tr[{i}]/td[@class="hprt-table-cell -first hprt-table-cell-roomtype droom_seperator"]//div[@class="hprt-roomtype-block"]//div[@class="hprt-facilities-facility"]')])

## Servicios por habitacion

In [137]:
services_h = []
for i in list_services:
    if i != []:
        services_h.append( ' - '.join(i) )

In [138]:
services_h

['16 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis',
 '19 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis',
 '22 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis']

## Capacidad y precio

In [118]:
cp = [i.text for i in driver.find_elements_by_xpath('//tbody/tr//span[@class="bui-u-sr-only"]')]

capacidad = cp[::2]
precio = cp[1::2]

In [121]:
capacidad

['Capacidad: de 1 a 1 personas',
 'Capacidad: de 1 a 2 personas',
 'Capacidad: de 1 a 1 personas',
 'Capacidad: de 1 a 2 personas',
 'Capacidad: de 1 a 3 personas',
 'Capacidad: de 1 a 2 personas',
 'Capacidad: de 1 a 3 personas',
 'Capacidad: de 1 a 4 personas']

In [120]:
precio

['Precio US$2.097',
 'Precio US$2.566',
 'Precio US$2.207',
 'Precio US$2.677',
 'Precio US$3.429',
 'Precio US$2.743',
 'Precio US$3.494',
 'Precio US$3.868']

In [134]:
logic_numbers = [re.sub(r'[A-Za-z]+: de \d a','',i).replace(' personas','').strip() for i in capacidad]

In [144]:
logic_numbers

['1', '2', '1', '2', '3', '2', '3', '4']

In [161]:
n_habitaciones = 0
for idx, n  in enumerate(logic_numbers):
    if logic_numbers[idx] == 0 or len(logic_numbers) -1 == idx:
        print(f'{habitaciones[n_habitaciones]} - {capacidad[idx]} - {camas_types[n_habitaciones]} - {services_h[n_habitaciones]} - {precio[idx]}')
    elif logic_numbers[idx] > logic_numbers[idx+1]:
        print(f'{habitaciones[n_habitaciones]} - {capacidad[idx]} - {camas_types[n_habitaciones]} - {services_h[n_habitaciones]} - {precio[idx]}')
        n_habitaciones += 1
    else:
        print(f'{habitaciones[n_habitaciones]} - {capacidad[idx]} - {camas_types[n_habitaciones]} - {services_h[n_habitaciones]} - {precio[idx]}')
        

Habitación Doble - 2 camas - Capacidad: de 1 a 1 personas - 2 camas individuales - 16 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis - Precio US$2.097
Habitación Doble - 2 camas - Capacidad: de 1 a 2 personas - 2 camas individuales - 16 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis - Precio US$2.566
Habitación Triple - Capacidad: de 1 a 1 personas - 1 cama individual y 1 cama doble - 19 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis - Precio US$2.207
Habitación Triple - Capacidad: de 1 a 2 personas - 1 cama individual y 1 cama doble - 19 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis - Precio US$2.677
Habitación Triple - Capacidad: de 1 a 3 personas - 1 cama individual y 1 cama doble - 19 m² - Vistas a la ciudad - Aire acondicionado - Baño privado - TV de pantalla plana - WiFi gratis - Pr

# Nombres de las etapas

In [9]:
r = requests.get('https://viajecaminodesantiago.com/camino-frances')
soup = BeautifulSoup(r.text, 'lxml')

In [14]:
names = [j.strip() for i in soup.find_all('div',attrs={'class':"vcs_stage"}) for j in i.find('span',attrs={'class':"vcs_stage_title"})]

In [15]:
names

['Saint Jean Pied de Port - Roncesvalles',
 'Roncesvalles - Zubiri',
 'Zubiri - Pamplona',
 'Pamplona - Puente la Reina',
 'Puente la Reina - Estella',
 'Estella - Los Arcos',
 'Los Arcos - Logroño',
 'Etapa 8: Logroño - Nájera (29 km)',
 'Nájera - Santo Domingo de la Calzada',
 'Santo Domingo de la Calzada - Belorado',
 'Belorado - San Juan de Ortega',
 'San Juan de Ortega - Burgos',
 'Burgos - Hornillos del Camino',
 'Hornillos del Camino - Castrojeriz',
 'Castrojeriz - Frómista',
 'Frómista - Carrión de los Condes',
 'Carrión de los Condes - Terradillo de los Templarios',
 'Terradillo de los Templarios - Bercianos del Real Camino',
 'Bercianos del Real Camino - Mansilla de las Mulas',
 'Mansilla de las Mulas - León',
 'León - Villadangos del Páramo',
 'Villadangos del Páramo - Astorga',
 'Astorga - Rabanal del Camino',
 'Rabanal del Camino - Molinaseca',
 'Molinaseca - Villafranca del Bierzo',
 'Villafranca del Bierzo - O Cebreiro',
 'O Cebreiro - Triacastela',
 'Triacastela - Sarri

## Buscar las etapas del camino

In [17]:
nombre = 'Saint Jean Pied de Port – Roncesvalles'

In [16]:
input_box = driver.find_element_by_xpath('//input[@id="ss"]')

In [18]:
input_box.click()
input_box.send_keys(nombre)

In [23]:
input_box.send_keys(Keys.ENTER)

In [ ]:
def get_main_pages(nombres):
    for nane in nombres:
        

# Close

In [193]:
with open('/home/cesar/Documents/job/web_scraping/javier/hoteles/list_of_links.dat','rb') as f:
    a = pickle.load(f)

In [199]:
url = a[0]['Link'][0]

# Test

In [184]:
driver.find_element_by_xpath('//td[@class="bui-calendar__date bui-calendar__date--today"]')

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="ed178f57-d997-42af-8493-5ece21e95fad", element="67c24bcf-0689-45a0-8afe-96906615a612")>

In [186]:
driver.find_element_by_xpath('//div[@class="sb-date-field__display"][@data-placeholder="Fecha de salida"]').click()

In [190]:
driver.find_element_by_xpath('//td[@class="bui-calendar__date bui-calendar__date--selected"]//span[@aria-label="30 abril 2022"]').click()

In [192]:
driver.find_element_by_xpath('//button[@class="sb-searchbox__button "]').click()

In [204]:
[i.text for i in driver.find_elements_by_xpath('//div[@class="_e3ed6b426 _425b57dc7 d79b273389 _b7ed60b59"]/div[@class="_361c04b9c _729127938 _78cd05546"]//div[@class="_a11e76d75 c5cf92dd20"]')]

['Cama en habitación compartida mixta de 10 camas',
 'Habitación Doble - 1 o 2 camas',
 'Habitación Doble - 2 camas']

In [234]:
[i.get_attribute('aria-label') for i in driver.find_elements_by_xpath('//div[@class="_361c04b9c _729127938 _10eb43c0c"]//div[@class="cea9cafb89"]')]

['1 adulto', '2 adultos', '2 adultos']

In [221]:
servicios = '  /  '.join([i.text.replace('\n',' - ') for i in driver.find_elements_by_xpath('//section[@id="hp_facilities_box"]//div[@class="hotel-facilities__list"]//div[@class="bui-spacer--large"]')])
servicios

'Baño - Papel higiénico  /  Habitación - Ropa de cama  /  Exteriores - Terraza  /  Mascotas - Se admiten bajo petición. Gratis.  /  Actividades - Sala de juegos  /  Internet - Hay conexión a internet Wi-Fi disponible en algunas habitaciones. Gratis.  /  Aparcamiento - Hay parking gratis público en las inmediaciones. No se puede reservar. - Parking en la calle  /  Servicios - Zona TV / salón de uso compartido  /  General - Prohibido fumar en todo el alojamiento - Calefacción - Habitaciones familiares - Habitaciones para no fumadores  /  Idiomas que se hablan - Inglés - Español - Francés - Tailandés'

In [225]:
[i.get_attribute('src') for i in driver.find_elements_by_xpath('//div[@class="clearfix bh-photo-grid bh-photo-grid--space-down fix-score-hover-opacity"]//div[@class="bh-photo-grid-thumbs bh-photo-grid-thumbs-s-full"]//img')]

['https://cf.bstatic.com/xdata/images/hotel/max300/181482772.jpg?k=a697298618d864b59362650c123fbbecc7dd73034392ed6415848198c8309fe2&o=&hp=1',
 'https://cf.bstatic.com/xdata/images/hotel/max300/181265215.jpg?k=6ac2d6de6087ad34d33282ebdd57e8ba8540bbea684fa9fe59366b3f16b1d1d5&o=&hp=1',
 'https://cf.bstatic.com/xdata/images/hotel/max300/181260547.jpg?k=738a8aaf195e99492f39b15a6eb47da1ae4012bd488fa41f30662bc15940fe5a&o=&hp=1',
 'https://cf.bstatic.com/xdata/images/hotel/max300/305940927.jpg?k=287a1fe43d6b93910d60eb7023db3c9e235f258153332ef83e45d2d4d17158f5&o=&hp=1',
 'https://cf.bstatic.com/xdata/images/hotel/max300/305921271.jpg?k=902cc63e688500d05032f5920e0dcc01a92fc4915ad553d8e2cb5e03e2341eb6&o=&hp=1']

In [233]:
[i.get_attribute('src') for i in driver.find_elements_by_xpath('//div[@aria-hidden="true"]/a//img')]

['https://cf.bstatic.com/xdata/images/hotel/max1024x768/310159470.jpg?k=b69d43ded03a20928f66ba215baade84f59cc42e5405594c5ff574854c685fe5&o=&hp=1',
 'https://cf.bstatic.com/xdata/images/hotel/max500/310159510.jpg?k=89190e7e0ed68c03d2ad0ed233d3022eab45261ae3440871a09e2317903fb289&o=&hp=1',
 'https://cf.bstatic.com/xdata/images/hotel/max500/305932953.jpg?k=72e255acc5bd4fe9f937fdef5652d835cb88bd0ca88897a4966d3ac12e37146e&o=&hp=1']

In [237]:
driver.find_element_by_xpath('//div[@class="hp_desc_main_content"]').text.replace('\n','')

'El Gîte BIDEAN se encuentra en Saint-Jean-Pied-de-Port y ofrece salón compartido, terraza y WiFi gratuita.Este bed and breakfast sirve un desayuno continental.El Gîte BIDEAN está a 42 km de Guéthary y a 27 km de Aïnhoa. El aeropuerto más cercano es el de Biarritz, ubicado a 53 km.A las parejas les encanta la ubicación — Le han puesto un 9,7 para viajes de dos personas.Gîte BIDEAN recibe clientes de Booking.com desde el 7 feb 2019.Marca/cadena de hotel:'

In [248]:
driver.get('https://www.booking.com/hotel/es/tuapartamento-plural-edificio-singular.es.html?label=gen173nr-1DCAEoggI46AdIM1gEaPEBiAEBmAEKuAEZyAEP2AED6AEBiAIBqAIDuAKfic6RBsACAdICJGJmOWFhZmQ5LTYyMDUtNGRlMC05MTliLWFkNTY4MjI2ZjgwONgCBOACAQ&sid=d100c388d933cb8f24019ed39aaaa276&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=10&hapos=235&sr_order=popularity&srpvid=5a6c73f598cc0194&srepoch=1647621025&from=searchresults#hotelTmpl')

In [241]:
driver.find_element_by_xpath('//h2[@id="hp_hotel_name"]').text

'Hostal o pensión La Villa Esponda'

In [268]:
driver.find_element_by_xpath('//div[@class="hp_desc_main_content"]').text.replace('\n','')

'¡Puedes conseguir un descuento Genius en La Villa Esponda! Para ahorrar en este alojamiento, solo tienes que iniciar sesión.La Villa Esponda, edificada en 1896, está ubicada en Saint-Jean-Pied-de-Port, frente a la muralla. Ofrece habitaciones con ventanas de doble acristalamiento, WiFi gratuita y aparcamiento gratuito.Las habitaciones y los apartamentos disponen de TV y aire acondicionado. Los apartamentos tienen cocina pequeña con mesa de comedor. Todos los alojamientos disponen de baño privado con secador de pelo. Las habitaciones tienen vistas a la muralla o a las montañas y a los viñedos de Irouléguy.A solo 2 minutos a pie, hay varios bares y restaurantes donde se sirven desayunos.El centro histórico de la fortaleza se halla a solo 50 metros. El aeropuerto de Anglet está a 53 km. Hay aparcamiento público gratuito en el establecimiento. Las plazas de aparcamiento son limitadas.Ver más'

In [264]:
driver.find_element_by_xpath('//div[@class="hp-description"]').text

'¡Puedes conseguir un descuento Genius en La Villa Esponda! Para ahorrar en este alojamiento, solo tienes que iniciar sesión.\nLa Villa Esponda, edificada en 1896, está ubicada en Saint-Jean-Pied-de-Port, frente a la muralla. Ofrece habitaciones con ventanas de doble acristalamiento, WiFi gratuita y aparcamiento gratuito.\nLas habitaciones y los apartamentos disponen de TV y aire acondicionado. Los apartamentos tienen cocina pequeña con mesa de comedor. Todos los alojamientos disponen de baño privado con secador de pelo. Las habitaciones tienen vistas a la muralla o a las montañas y a los viñedos de Irouléguy.\nA solo 2 minutos a pie, hay varios bares y restaurantes donde se sirven desayunos.\nEl centro histórico de la fortaleza se halla a solo 50 metros. El aeropuerto de Anglet está a 53 km. Hay aparcamiento público gratuito en el establecimiento. Las plazas de aparcamiento son limitadas.\n\nVer más'

In [245]:
element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h2[@id="hp_hotel_name"]'))
    ).text

In [47]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [54]:
driver.execute_script("window.scrollTo(0, 1800)")

In [64]:
driver.get('https://www.booking.com/hotel/fr/la-villa-esponda.es.html?label=gen173nr-1BCAEoggI46AdIM1gEaPEBiAEBmAEKuAEZyAEP2AEB6AEBiAIBqAIDuAKfic6RBsACAdICJGJmOWFhZmQ5LTYyMDUtNGRlMC05MTliLWFkNTY4MjI2ZjgwONgCBeACAQ&sid=d100c388d933cb8f24019ed39aaaa276&aid=304142&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=3&hapos=3&sr_order=popularity&srpvid=1c7c73a884b101f8&srepoch=1647620817&from=searchresults#hotelTmpl')

time.sleep(3)
title =  WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//h2[@id="hp_hotel_name"]'))
                        ).text
driver.implicitly_wait(9)
driver.execute_script("window.scrollTo(0, 1800)")

In [65]:
title

'Hostal o pensión La Villa Esponda'

In [36]:
with open('data_info.dat','rb') as f:
    l = pickle.load(f)

In [39]:
l[0]['Habitaciones'] = None

In [43]:
pd.DataFrame(l[0], index=[0])

,Title,Habitaciones,Services,Location,Iamges,City,Description
0,Bed & breakfast Gîte BIDEAN,None,Baño - Papel higiénico / Habitación - Ropa d...,"11 rue d'Espagne, 64220 Saint-Jean-Pied-de-Por...",https://t-cf.bstatic.com/xdata/images/hotel/ma...,Saint Jean Pied de Port - Roncesvalles,El Gîte BIDEAN se encuentra en Saint-Jean-Pied...


In [40]:
del l[0]['Capcidad']

In [11]:
data = l

In [12]:
for idx, i in enumerate(data):
    try:
        data[idx] = pd.DataFrame(i)
    except ValueError as e:
        data[idx] = pd.DataFrame.from_dict(i,orient='index').transpose()

In [13]:
data

[Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Services, Capcidad, Location, Iamges, City, Description]
 Index: [],
 Empty DataFrame
 Columns: [Title, Habitaciones, Service

In [9]:
pd.concat(data)

,Title,Habitaciones,Services,Capcidad,Location,Iamges,City,Description


In [74]:
df = pd.read_csv('./hoteles_data.xlsx',index_col=['Unnamed: 0'])

In [75]:
df.drop_duplicates(inplace=True)

In [79]:
df = df.reset_index().drop(columns=['index'])

In [84]:
test_string = df['Location'][0]
test_string

"11 rue d'Espagne, 64220 Saint-Jean-Pied-de-Port, Francia"

In [123]:
def split_strings(x,column):
    if len(x.split(',')) == 3:
        if column == 'calle':
            return x.split(',')[0]
        if column == 'postal_code':
            return x.split(',')[1].strip().split(' ')[0]
        if column == 'porvincia':
            return x.split(',')[-1].strip()

    elif len(x.split(',')) == 4:
        if column == 'calle':
            return x.split(',')[0]
        if column == 'postal_code':
            return x.split(',')[2].strip().split(' ')[0]
        if column == 'porvincia':
            return x.split(',')[-1].strip()


In [129]:
df['Codigo_postal'] = df['Location'].apply(lambda x : split_strings(x,'postal_code'))

In [130]:
df['Calle'] = df['Location'].apply(lambda x : split_strings(x,'calle'))

In [131]:
df['Porvincia'] = df['Location'].apply(lambda x : split_strings(x,'porvincia'))

In [138]:
test_string = df.Services[0]
test_string 

'Baño - Papel higiénico  /  Habitación - Ropa de cama  /  Exteriores - Terraza  /  Mascotas - Se admiten bajo petición. Gratis.  /  Actividades - Sala de juegos  /  Internet - Hay conexión a internet Wi-Fi disponible en algunas habitaciones. Gratis.  /  Aparcamiento - Hay parking gratis público en las inmediaciones. No se puede reservar. - Parking en la calle  /  Servicios - Zona TV / salón de uso compartido  /  General - Prohibido fumar en todo el alojamiento - Calefacción - Habitaciones familiares - Habitaciones para no fumadores  /  Idiomas que se hablan - Inglés - Español - Francés - Tailandés'

In [151]:
def all_services(df):
    data_services = []
    for x in df:
        if type(x) == str:
            x = x.split(' / ')
            x = [i.split(' - ')[0].strip() for i in x]
            for serv in x:
                data_services.append(serv)
    return set(data_services)

In [153]:
services = all_services(df.Services)

In [155]:
services = list(services)

In [205]:
def split_services_string(x : str, service):
    if type(x) == str:
        x = x.split(' / ')
        for i in x:
            ser = i.split(' - ')
            if ser[0].strip() == service:
                return ' '.join(ser[1:])
    

In [187]:
test = df[['Services']]

In [191]:
test.Services

0       Baño - Papel higiénico  /  Habitación - Ropa d...
1       Baño - Papel higiénico  /  Habitación - Ropa d...
2       Baño - Papel higiénico  /  Habitación - Ropa d...
3       Baño - Papel higiénico - Toallas - Bañera o du...
4       Baño - Papel higiénico - Toallas - Bañera o du...
                              ...                        
5708    Baño - Baño privado  /  Exteriores - Terraza -...
5709    Baño - Baño privado  /  Exteriores - Terraza -...
5710    Baño - Baño privado  /  Exteriores - Terraza -...
5711    Baño - Baño privado  /  Exteriores - Terraza -...
5712    Baño - Baño privado  /  Exteriores - Terraza -...
Name: Services, Length: 5713, dtype: object

In [234]:
for i in services:
    df[f'{i}'] = test.Services.apply(lambda x : split_services_string(x,i))

In [237]:
df.drop(columns=['Services'], inplace=True)

In [239]:
df.to_excel('final_df_hoteles.xlsx')